## SRJ - GS state

In [7]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, electrons, orbitals, adapt_it, shots = None):
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf")
    print(H)
    hf_state = qchem.hf_state(electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        print(qml.PauliX(i) for i in np.nonzero(hf_state)[0])
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                #print('Exc. dealing right now is', ash_excitation[i])
                #print('The params that are going in', params[i])
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                print('Single Exc. dealing right now is', ash_excitation[i])
                print('Single exc params that are going in', params[i])
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value


        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization



        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = 1e-12, options = {'disp': False, 'maxiter': 1e8})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)


        ostate = new_state(hf_state, ash_excitation, params)
        #print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params

## So if you want the state, return the ostate and not states




## starting from null state

In [ ]:
symbols  = [ 'H', 'H', 'H', 'H']
r_bohr = 1.8897259886 
coordinates = np.array([[0.0, 0.0,  0.0],
                     [0.0, 0.0, 1.0*r_bohr],
                     [0.0, 0.0, 2.0*r_bohr],
                     [0.0, 0.0, 3.0*r_bohr]])


electrons = 4
orbitals = 8
charge = 0


gs_state,params = ags_exact(symbols, coordinates, electrons, orbitals, shots = None, adapt_it=3) #1 is used for params
qubits = 8
H = (-0.3314776478540091 * I([0, 1, 2, 3, 4, 5, 6, 7]) + 0.18136486311337868 * Z(0) + 0.005562131543288114 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.005562131543288114 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.08792644810341417 * Z(2) + 0.06963751217552142 * (Z(0) @ Z(2)) + 0.017266562082446573 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.017266562082446573 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.010290903378955305 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.010290903378955305 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.07904410972743413 * Z(4) + 0.08454049632677563 * (Z(0) @ Z(4)) + -0.33461218695471684 * Z(6) + 0.10647068976730581 * (Z(0) @ Z(6)) + 0.18136486311337868 * Z(1) + 0.12432121277101203 * (Z(0) @ Z(1)) + 0.0203912989817354 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.0203912989817354 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.039345513502593965 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.039345513502593965 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.039345513502593965 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.039345513502593965 * (X(0) @ Y(1) @ Y(2) @ X(3)) + -0.010770994711108364 * (Y(0) @ X(1) @ X(3) @ Z(4) @ Z(5) @ Y(6)) + -0.010770994711108364 * (Y(0) @ Y(1) @ Y(3) @ Z(4) @ Z(5) @ Y(6)) + -0.010770994711108364 * (X(0) @ X(1) @ X(3) @ Z(4) @ Z(5) @ X(6)) + -0.010770994711108364 * (X(0) @ Y(1) @ Y(3) @ Z(4) @ Z(5) @ X(6)) + 0.005562131543288103 * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5)) + 0.020391298981735397 * (Z(0) @ Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5)) + 0.005562131543288103 * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5)) + 0.020391298981735397 * (Z(0) @ X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5)) + 0.026958030153271643 * (Y(0) @ X(1) @ X(4) @ Y(5)) + -0.026958030153271643 * (Y(0) @ Y(1) @ X(4) @ X(5)) + -0.026958030153271643 * (X(0) @ X(1) @ Y(4) @ Y(5)) + 0.026958030153271643 * (X(0) @ Y(1) @ Y(4) @ X(5)) + 0.010770994711108364 * (Y(0) @ X(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + -0.010770994711108364 * (Y(0) @ Y(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + -0.010770994711108364 * (X(0) @ X(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + 0.010770994711108364 * (X(0) @ Y(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + 0.024267373887137178 * (Y(0) @ X(1) @ X(6) @ Y(7)) + -0.024267373887137178 * (Y(0) @ Y(1) @ X(6) @ X(7)) + -0.024267373887137178 * (X(0) @ X(1) @ Y(6) @ Y(7)) + 0.024267373887137178 * (X(0) @ Y(1) @ Y(6) @ X(7)) + 0.02204895116824182 * (Y(0) @ Z(1) @ Z(3) @ Y(4)) + 0.02204895116824182 * (X(0) @ Z(1) @ Z(3) @ X(4)) + -0.011388106363309936 * (Y(0) @ Z(1) @ X(2) @ X(4) @ Z(5) @ Y(6)) + -0.024418228642946037 * (Y(0) @ Z(1) @ Y(2) @ Y(4) @ Z(5) @ Y(6)) + -0.0130301222796361 * (Y(0) @ Z(1) @ Y(2) @ X(4) @ Z(5) @ X(6)) + -0.0130301222796361 * (X(0) @ Z(1) @ X(2) @ Y(4) @ Z(5) @ Y(6)) + -0.024418228642946037 * (X(0) @ Z(1) @ X(2) @ X(4) @ Z(5) @ X(6)) + -0.011388106363309936 * (X(0) @ Z(1) @ Y(2) @ Y(4) @ Z(5) @ X(6)) + 0.08792644810341417 * Z(3) + 0.10898302567811538 * (Z(0) @ Z(3)) + -0.002451287918648047 * (Y(0) @ Z(1) @ Z(2) @ Y(4)) + -0.002451287918648047 * (X(0) @ Z(1) @ Z(2) @ X(4)) + 0.024500239086889863 * (Y(0) @ Y(3) @ Z(4) @ Y(5) @ Z(1) @ Y(2)) + 0.024500239086889863 * (Y(0) @ X(3) @ Z(4) @ X(5) @ Z(1) @ Y(2)) + 0.024500239086889863 * (X(0) @ Y(3) @ Z(4) @ Y(5) @ Z(1) @ X(2)) + 0.024500239086889863 * (X(0) @ X(3) @ Z(4) @ X(5) @ Z(1) @ X(2)) + 0.013240138894796055 * (Y(0) @ Z(1) @ Z(2) @ X(3) @ X(5) @ Y(6)) + 0.013240138894796055 * (Y(0) @ Z(1) @ Z(2) @ Y(3) @ Y(5) @ Y(6)) + 0.013240138894796055 * (X(0) @ Z(1) @ Z(2) @ X(3) @ X(5) @ X(6)) + 0.013240138894796055 * (X(0) @ Z(1) @ Z(2) @ Y(3) @ Y(5) @ X(6)) + 0.017266562082446563 * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + -0.02106189809006367 * (Z(0) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + 0.017266562082446563 * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + -0.02106189809006367 * (Z(0) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + -0.024628245258105988 * (Y(0) @ X(3) @ X(4) @ Z(5) @ Z(6) @ Y(7) @ Z(1) @ Z(2)) + 0.024628245258105988 * (Y(0) @ Y(3) @ X(4) @ Z(5) @ Z(6) @ X(7) @ Z(1) @ Z(2)) + 0.024628245258105988 * (X(0) @ X(3) @ Y(4) @ Z(5) @ Z(6) @ Y(7) @ Z(1) @ Z(2)) + -0.024628245258105988 * (X(0) @ Y(3) @ Y(4) @ Z(5) @ Z(6) @ X(7) @ Z(1) @ Z(2)) + 0.011234473981208727 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(6)) + 0.011234473981208727 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(6)) + -0.07904410972743411 * Z(5) + 0.11149852648004727 * (Z(0) @ Z(5)) + 0.0017156580798700712 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(5)) + 0.0017156580798700712 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(5)) + -0.0376583675377421 * (Y(0) @ Y(5) @ Z(6) @ Y(7) @ Z(1) @ Y(2)) + -0.0376583675377421 * (Y(0) @ X(5) @ Z(6) @ X(7) @ Z(1) @ Y(2)) + -0.0376583675377421 * (X(0) @ Y(5) @ Z(6) @ Y(7) @ Z(1) @ X(2)) + -0.0376583675377421 * (X(0) @ X(5) @ Z(6) @ X(7) @ Z(1) @ X(2)) + -0.010242352622888933 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(6) @ Y(7)) + 0.010242352622888933 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ X(6) @ X(7)) + 0.010242352622888933 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Y(6) @ Y(7)) + -0.010242352622888933 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(6) @ X(7)) + -0.33461218695471684 * Z(7) + 0.130738063654443 * (Z(0) @ Z(7)) + 0.021476826604097658 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(7)) + 0.021476826604097658 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(7)) + 0.06963751217552142 * (Z(1) @ Z(3)) + -0.010290903378955305 * (Z(1) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + -0.010290903378955305 * (Z(1) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + 0.08454049632677563 * (Z(1) @ Z(5)) + 0.10647068976730581 * (Z(1) @ Z(7)) + 0.10898302567811538 * (Z(1) @ Z(2)) + -0.002451287918648047 * (Y(1) @ Z(3) @ Z(4) @ Y(5)) + -0.002451287918648047 * (X(1) @ Z(3) @ Z(4) @ X(5)) + 0.024500239086889867 * (Y(1) @ X(2) @ X(3) @ Y(4)) + -0.024500239086889867 * (Y(1) @ Y(2) @ X(3) @ X(4)) + -0.024500239086889867 * (X(1) @ X(2) @ Y(3) @ Y(4)) + 0.024500239086889867 * (X(1) @ Y(2) @ Y(3) @ X(4)) + 0.013240138894796055 * (Y(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ Y(7)) + 0.013240138894796055 * (Y(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ Y(7)) + 0.013240138894796055 * (X(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ X(7)) + 0.013240138894796055 * (X(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ X(7)) + -0.02106189809006367 * (Z(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.02106189809006367 * (Z(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.024628245258105988 * (Y(1) @ X(2) @ X(5) @ Y(6)) + 0.024628245258105988 * (Y(1) @ Y(2) @ X(5) @ X(6)) + 0.024628245258105988 * (X(1) @ X(2) @ Y(5) @ Y(6)) + -0.024628245258105988 * (X(1) @ Y(2) @ Y(5) @ X(6)) + 0.02204895116824182 * (Y(1) @ Z(2) @ Z(4) @ Y(5)) + 0.02204895116824182 * (X(1) @ Z(2) @ Z(4) @ X(5)) + -0.011388106363309936 * (Y(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ Y(7)) + -0.024418228642946037 * (Y(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ Y(7)) + -0.0130301222796361 * (Y(1) @ Z(2) @ Y(3) @ X(5) @ Z(6) @ X(7)) + -0.0130301222796361 * (X(1) @ Z(2) @ X(3) @ Y(5) @ Z(6) @ Y(7)) + -0.024418228642946037 * (X(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ X(7)) + -0.011388106363309936 * (X(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ X(7)) + 0.11149852648004727 * (Z(1) @ Z(4)) + 0.0017156580798700712 * (Y(1) @ Z(2) @ Z(3) @ Y(5)) + 0.0017156580798700712 * (X(1) @ Z(2) @ Z(3) @ X(5)) + -0.0376583675377421 * (Y(1) @ Y(4) @ Z(5) @ Y(6) @ Z(2) @ Y(3)) + -0.0376583675377421 * (Y(1) @ X(4) @ Z(5) @ X(6) @ Z(2) @ Y(3)) + -0.0376583675377421 * (X(1) @ Y(4) @ Z(5) @ Y(6) @ Z(2) @ X(3)) + -0.0376583675377421 * (X(1) @ X(4) @ Z(5) @ X(6) @ Z(2) @ X(3)) + 0.010242352622888933 * (Y(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ Y(7)) + 0.010242352622888933 * (Y(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ Y(7)) + 0.010242352622888933 * (X(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ X(7)) + 0.010242352622888933 * (X(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ X(7)) + 0.011234473981208727 * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Z(7)) + 0.011234473981208727 * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Z(7)) + 0.130738063654443 * (Z(1) @ Z(6)) + 0.021476826604097658 * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Z(6)) + 0.021476826604097658 * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Z(6)) + 0.0776203766136051 * (Z(2) @ Z(4)) + 0.0898249970095836 * (Z(2) @ Z(6)) + 0.11340654529950416 * (Z(2) @ Z(3)) + -0.0010141247044787717 * (Y(2) @ Z(4) @ Z(5) @ Y(6)) + -0.0010141247044787717 * (X(2) @ Z(4) @ Z(5) @ X(6)) + 0.03432067766799027 * (Y(2) @ X(3) @ X(4) @ Y(5)) + -0.03432067766799027 * (Y(2) @ Y(3) @ X(4) @ X(5)) + -0.03432067766799027 * (X(2) @ X(3) @ Y(4) @ Y(5)) + 0.03432067766799027 * (X(2) @ Y(3) @ Y(4) @ X(5)) + -0.0010141247044787717 * (Z(2) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + -0.0010141247044787717 * (Z(2) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + 0.026161331391178138 * (Y(2) @ X(3) @ X(6) @ Y(7)) + -0.026161331391178138 * (Y(2) @ Y(3) @ X(6) @ X(7)) + -0.026161331391178138 * (X(2) @ X(3) @ Y(6) @ Y(7)) + 0.026161331391178138 * (X(2) @ Y(3) @ Y(6) @ X(7)) + -0.025545242831226625 * (Y(2) @ Z(3) @ Z(5) @ Y(6)) + -0.025545242831226625 * (X(2) @ Z(3) @ Z(5) @ X(6)) + 0.11194105428159537 * (Z(2) @ Z(5)) + -0.0007036287022399996 * (Y(2) @ Z(3) @ Z(4) @ Y(6)) + -0.0007036287022399996 * (X(2) @ Z(3) @ Z(4) @ X(6)) + -0.024841614128986618 * (Y(2) @ Y(5) @ Z(6) @ Y(7) @ Z(3) @ Y(4)) + -0.024841614128986618 * (Y(2) @ X(5) @ Z(6) @ X(7) @ Z(3) @ Y(4)) + -0.024841614128986618 * (X(2) @ Y(5) @ Z(6) @ Y(7) @ Z(3) @ X(4)) + -0.024841614128986618 * (X(2) @ X(5) @ Z(6) @ X(7) @ Z(3) @ X(4)) + 0.11598632840076173 * (Z(2) @ Z(7)) + -0.02338450515476927 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Z(7)) + -0.02338450515476927 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Z(7)) + 0.0776203766136051 * (Z(3) @ Z(5)) + 0.0898249970095836 * (Z(3) @ Z(7)) + 0.11194105428159537 * (Z(3) @ Z(4)) + -0.0007036287022399996 * (Y(3) @ Z(5) @ Z(6) @ Y(7)) + -0.0007036287022399996 * (X(3) @ Z(5) @ Z(6) @ X(7)) + -0.024841614128986618 * (Y(3) @ X(4) @ X(5) @ Y(6)) + 0.024841614128986618 * (Y(3) @ Y(4) @ X(5) @ X(6)) + 0.024841614128986618 * (X(3) @ X(4) @ Y(5) @ Y(6)) + -0.024841614128986618 * (X(3) @ Y(4) @ Y(5) @ X(6)) + -0.025545242831226625 * (Y(3) @ Z(4) @ Z(6) @ Y(7)) + -0.025545242831226625 * (X(3) @ Z(4) @ Z(6) @ X(7)) + 0.11598632840076173 * (Z(3) @ Z(6)) + -0.023384505154769267 * (Y(3) @ Z(4) @ Z(5) @ Y(7)) + -0.023384505154769267 * (X(3) @ Z(4) @ Z(5) @ X(7)) + 0.07943849146260731 * (Z(4) @ Z(6)) + 0.11685144248370052 * (Z(4) @ Z(5)) + 0.04061611914426471 * (Y(4) @ X(5) @ X(6) @ Y(7)) + -0.04061611914426471 * (Y(4) @ Y(5) @ X(6) @ X(7)) + -0.04061611914426471 * (X(4) @ X(5) @ Y(6) @ Y(7)) + 0.04061611914426471 * (X(4) @ Y(5) @ Y(6) @ X(7)) + 0.12005461060687202 * (Z(4) @ Z(7)) + 0.07943849146260731 * (Z(5) @ Z(7)) + 0.12005461060687202 * (Z(5) @ Z(6)) + 0.14526148569746633 * (Z(6) @ Z(7))
)
null_state = np.zeros(qubits,int)
print('Null state before loop', null_state)

dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def es_state(null_state, ash_excitation, params):
    qml.BasisState(null_state, wires=range(qubits)) #Applying the NULL STATE
    #print('Null state is',[qml.PauliX(i) for i in np.nonzero(null_state)[0]])
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print('Exc. dealing right now is', ash_excitation[i])
            print('The params that are going in', params[i])
            qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc. dealing right now is', ash_excitation[i])
            print('Single exc params that are going in', params[i])
            qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
    return qml.state()

zstate = es_state(null_state, ash_excitation, params)
print('kstate:', zstate)

print('Going to calculate exp values')

@qml.qnode(dev)
def dummy():
    qml.StatePrep(zstate, wires=range(qubits))
    return qml.expval(H)

dummy()

-0.3314776478540091 * I([0, 1, 2, 3, 4, 5, 6, 7]) + 0.18136486311337868 * Z(0) + 0.005562131543288114 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.005562131543288114 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.08792644810341417 * Z(2) + 0.06963751217552142 * (Z(0) @ Z(2)) + 0.017266562082446573 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.017266562082446573 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.010290903378955305 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.010290903378955305 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.07904410972743413 * Z(4) + 0.08454049632677563 * (Z(0) @ Z(4)) + -0.33461218695471684 * Z(6) + 0.10647068976730581 * (Z(0) @ Z(6)) + 0.18136486311337868 * Z(1) + 0.12432121277101203 * (Z(0) @ Z(1)) + 0.0203912989817354 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.0203912989817354 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.039345513502593965 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.039345513502593965 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.039345513502593965 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.03934551

/Users/s.poyyapakkam/miniconda3/envs/sri/lib/python3.12/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Highest gradient excitation is [2, 3, 4, 5]
Current parameters: [-0.34549402]
Current cost: -2.1224998899999097

Current parameters: [-0.34549401]
Current cost: -2.1224998899999132

Final updated parameters: [-0.34549401]
Final cost: -2.1224998899999132
<generator object ags_exact.<locals>.new_state.<locals>.<genexpr> at 0x108444520>
The adapt iteration now is 1
Highest gradient excitation is [0, 3, 4, 7]
Current parameters: [-0.34549401  0.17199106]
Current cost: -2.132932679985641

Current parameters: [-0.371204    0.17451109]
Current cost: -2.1330713159332353

Current parameters: [-0.37163179  0.17455302]
Current cost: -2.133071353098797

Final updated parameters: [-0.37163179  0.17455302]
Final cost: -2.133071353098797
<generator object ags_exact.<locals>.new_state.<locals>.<genexpr> at 0x11a7268e0>
The adapt iteration now is 2
Highest gradient excitation is [1, 2, 5, 6]
Current parameters: [-0.37163179  0.17455303  0.17638609]
Current cost: -2.1440203206294295

Current parameters:

-2.1441833159359365

In [5]:
print(params)
print(ash_excitation)
print(zstate)

#2.2931014123813886

[-0.39987225  0.17934177  0.1793417 ]
[[2, 3, 4, 5], [0, 3, 4, 7], [1, 2, 5, 6]]
[-3.42113883e-49+0.00000000e+00j  0.00000000e+00+1.25289107e-32j
  0.00000000e+00+1.07129853e-33j -3.85185989e-34+0.00000000e+00j
  0.00000000e+00+1.54074396e-33j -3.81314675e-33+0.00000000e+00j
 -1.09846032e-19+0.00000000e+00j  0.00000000e+00+5.90313842e-18j
  0.00000000e+00+3.08148791e-33j -4.59988025e-19+0.00000000e+00j
  8.47409176e-33+0.00000000e+00j  0.00000000e+00-9.54097912e-18j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00-1.53329342e-19j
  0.00000000e+00-1.54882905e-17j  7.85958491e-03+0.00000000e+00j
  0.00000000e+00+6.80094012e-34j  4.52593537e-33+0.00000000e+00j
  1.34815096e-33+0.00000000e+00j  0.00000000e+00-6.74075481e-34j
  5.58820610e-33+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+4.97106394e-18j  1.75753651e-18+0.00000000e+00j
  2.24641672e-33+0.00000000e+00j  0.00000000e+00+1.36845553e-48j
  0.00000000e+00-1.92592994e-34j -9.62964972e-35+0.00000000e+00j
  0.00000

## Edited code

In [6]:
import os
from time import time
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy



def inite(elec,orb):
    config=[]
    list1=[]
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)
                
            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:    
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)

                        list1.append(config)
                        config=[]
    return list1

def ee_exact(symbols, coordinates, electrons, charge,params,zstate,shots=0):

    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method='pyscf')
    hf_state = qml.qchem.hf_state(electrons, qubits)
    print('HF state:', hf_state)
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)

    null_state = np.zeros(qubits,int)
    print('Null state is', null_state)
    list1 = inite(electrons,qubits)
    values =[]
    for t in range(1):
        if shots==0:
            dev = qml.device("lightning.qubit", wires=qubits)
        else:

            dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, s_wires, d_wires, hf_state):
            for w in occ:
                qml.X(wires=w)
            qml.StatePrep(zstate, wires=range(qubits))
            #qml.UCCSD(params, wires, s_wires, d_wires, null_state)
            return qml.expval(H)
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2,wires, s_wires, d_wires, hf_state):
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            qml.StatePrep(zstate, wires=range(qubits))
            #qml.UCCSD(params, wires, s_wires, d_wires, null_state)
            return qml.expval(H)
        #final M matrix
        M = np.zeros((len(list1),len(list1)))
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i == j:
                    M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, zstate)
        print("diagonal parts done")
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i!=j:
                    Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, zstate)
                    M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
        print("off diagonal terms done")
        #ERROR:not subtracting the gs energy
        eig,evec=np.linalg.eig(M)
        values.append(np.sort(eig))
    return values

symbols  = [ 'H', 'H', 'H', 'H']
r_bohr = 1.8897259886 
coordinates = np.array([[0.0, 0.0,  0.0],
                     [0.0, 0.0, 1.0*r_bohr],
                     [0.0, 0.0, 2.0*r_bohr],
                     [0.0, 0.0, 3.0*r_bohr]])


electrons = 4
orbitals = 8
charge = 0


gs_state,params = ags_exact(symbols, coordinates, electrons, orbitals, shots = None, adapt_it=3) #1 is used for params



print('The params after GS is',params)
print('GS state is', gs_state)



#params_exact=params

eig = ee_exact(symbols, coordinates, electrons, charge,params,gs_state)
print('exact eigenvalues:\n', eig)

-0.3314776478540091 * I([0, 1, 2, 3, 4, 5, 6, 7]) + 0.18136486311337868 * Z(0) + 0.005562131543288114 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.005562131543288114 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.08792644810341417 * Z(2) + 0.06963751217552142 * (Z(0) @ Z(2)) + 0.017266562082446573 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.017266562082446573 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.010290903378955305 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.010290903378955305 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.07904410972743413 * Z(4) + 0.08454049632677563 * (Z(0) @ Z(4)) + -0.33461218695471684 * Z(6) + 0.10647068976730581 * (Z(0) @ Z(6)) + 0.18136486311337868 * Z(1) + 0.12432121277101203 * (Z(0) @ Z(1)) + 0.0203912989817354 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.0203912989817354 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.039345513502593965 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.039345513502593965 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.039345513502593965 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.03934551

KeyboardInterrupt: 

## Correct answer

In [ ]:
[-0.5246155 , -0.1627531 ,  0.49505785]